# Top Manufacturers and Aircraft Types Over Time

This notebook calculates and exports pie charts of:

- Top manufacturers over all time
- Top aircraft types over all time

To use it, set the top 'N' values and export format for charts in the first code cell before running the notebook.

In [ ]:
# Top "N" manufacturers and aircraft types to include
top_n_manufacturers = 10
top_n_models = 10

# Export format for the charts:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "sightings.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case='upper'))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["FLIGHT_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found"
    raise ValueError(message)

In [ ]:
# Calculate the manufacturer and model counts
manufacturer_counts = df["Manufacturer"].value_counts().nlargest(top_n_manufacturers)
model_counts = df["Model"].value_counts().nlargest(top_n_models)

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Create a Pandas Excel writer
output_path = export_folder_path / f"Top-Manufaturers-And-Aircraft.xlsx"

with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    # Daily data
    manufacturer_counts.to_excel(writer, sheet_name="Manufacturers", index=True)
    
    # Monthly data
    model_counts.to_excel(writer, sheet_name="Aircraft", index=True)

In [ ]:
import matplotlib.pyplot as plt

# Create labels for the legend
labels = manufacturer_counts.index
sizes = manufacturer_counts.values
total = sizes.sum()
legend_labels = [f"{label} ({size / total:.1%})" for label, size in zip(labels, sizes)]

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Plot pie chart without labels
wedges, texts = ax.pie(sizes, startangle=140)

# Add title
ax.set_title(f"Top {top_n_manufacturers} Aircraft Manufacturers")

# Add legend below the chart
ax.legend(
    wedges,
    legend_labels,
    title="Manufacturers",
    loc="lower center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=2,  # Change number of columns to fit nicely
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"Top-Aircraft-Models.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"Top-Aircraft-Models.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()

In [ ]:
# Create labels for the legend
labels = model_counts.index
sizes = model_counts.values
total = sizes.sum()
legend_labels = [f"{label} ({size / total:.1%})" for label, size in zip(labels, sizes)]

# Plot pie chart without labels
fig, ax = plt.subplots(figsize=(8, 8))
wedges, texts = ax.pie(sizes, startangle=140)

# Set the title
ax.set_title(f"Top {top_n_models} Aircraft Models")

# Create the legend
ax.legend(
    wedges,
    legend_labels,
    title="Aircraft Types",
    loc="lower center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=2,
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"Top-Aircraft-Manufacturers.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"Top-Aircraft-Manufacturers.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()